In [1]:
PATH_TO_DATA = './sarcasm/train-balanced-sarcasm.csv'

In [19]:
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

## Plot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import matplotlib as plt

# NLTK
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ashwi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv(PATH_TO_DATA, usecols=['label','comment'])

In [4]:
df.head()

,label,comment
0,0,NC and NH.
1,0,You do know west teams play against west teams...
2,0,"They were underdogs earlier today, but since G..."
3,0,"This meme isn't funny none of the ""new york ni..."
4,0,I could use one of those tools.


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010826 entries, 0 to 1010825
Data columns (total 2 columns):
label      1010826 non-null int64
comment    1010773 non-null object
dtypes: int64(1), object(1)
memory usage: 15.4+ MB


In [6]:
df.dropna(subset=['comment'], inplace=True)

In [7]:
df['label'].value_counts()

0    505405
1    505368
Name: label, dtype: int64

In [8]:
#train_texts, valid_texts, y_train, y_valid = train_test_split(train_df['comment'], train_df['label'], random_state=10, shuffle=True, stratify=train_df['label'])

In [9]:
#train_df.loc[train_df['label'] == 1, 'comment'].str.len().apply(np.log1p).hist(label='sarcastic', alpha=.5)
#train_df.loc[train_df['label'] == 0, 'comment'].str.len().apply(np.log1p).hist(label='normal', alpha=.5)

In [14]:
# text = text.translate(string.punctuation)
# text = text.lower().split()
stops = set(stopwords.words("english"))
stemmer = SnowballStemmer('english')
df['comment'] = df['comment'].apply(lambda x: ' '.join([word for word in x.lower().split() if word not in (stops)]))
df['comment'] = df['comment'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.lower().split() ]))

In [20]:
df.head(20)
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['comment'])

sequences = tokenizer.texts_to_sequences(df['comment'])
data = pad_sequences(sequences, maxlen=50)

In [21]:
print(data.shape)

(1010773, 50)


In [22]:
model_lstm = Sequential()
model_lstm.add(Embedding(20000, 100, input_length=50))
model_lstm.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_lstm.add(Dense(1, activation='sigmoid'))
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
model_lstm.fit(data, df['label'], validation_split=0.4, epochs=3)

Train on 606463 samples, validate on 404310 samples
Epoch 1/3
606463/606463 [==============================] - 2107s 3ms/step - loss: 0.5909 - acc: 0.6809 - val_loss: 0.5772 - val_acc: 0.6905
Epoch 2/3
606463/606463 [==============================] - 2141s 4ms/step - loss: 0.5542 - acc: 0.7128 - val_loss: 0.5735 - val_acc: 0.6947
Epoch 3/3
606463/606463 [==============================] - 2148s 4ms/step - loss: 0.5309 - acc: 0.7303 - val_loss: 0.5806 - val_acc: 0.6918


In [24]:
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_conv

In [ ]:
model_conv = create_conv_model()
model_conv.fit(data, df['label'], validation_split=0.4, epochs = 3)

Train on 606463 samples, validate on 404310 samples
Epoch 1/3
606463/606463 [==============================] - 1433s 2ms/step - loss: 0.6315 - acc: 0.6371 - val_loss: 0.6171 - val_acc: 0.6453
Epoch 2/3
606463/606463 [==============================] - 1454s 2ms/step - loss: 0.6074 - acc: 0.6611 - val_loss: 0.6204 - val_acc: 0.6440
Epoch 3/3
521472/606463 [========================>.....] - ETA: 3:03 - loss: 0.5834 - acc: 0.6808

In [ ]:
embeddings_index = dict()
f = open('glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [ ]:
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=50, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(100))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_glove.fit(data, df['label'], validation_split=0.4, epochs = 3)

In [ ]:
lstm_embds = model_lstm.layers[0].get_weights()[0]
conv_embds = model_conv.layers[0].get_weights()[0]
glove_emds = model_glove.layers[0].get_weights()[0]

In [ ]:
word_list = []
for word, i in tokenizer.word_index.items():
    word_list.append(word)

In [ ]:
def plot_words(data, start, stop, step):
    trace = go.Scatter(
        x = data[start:stop:step,0], 
        y = data[start:stop:step, 1],
        mode = 'markers',
        text= word_list[start:stop:step]
    )
    layout = dict(title= 't-SNE 1 vs t-SNE 2',
                  yaxis = dict(title='t-SNE 2'),
                  xaxis = dict(title='t-SNE 1'),
                  hovermode= 'closest')
    fig = dict(data = [trace], layout= layout)
    py.iplot(fig)

In [ ]:
lstm_tsne_embds = TSNE(n_components=2).fit_transform(lstm_embds)

In [ ]:
plot_words(lstm_tsne_embds, 0, 2000, 1)

In [ ]:
conv_tsne_embds = TSNE(n_components=2).fit_transform(conv_embds)

In [ ]:
plot_words(conv_tsne_embds, 0, 2000, 1)

In [ ]:
glove_tsne_embds = TSNE(n_components=2).fit_transform(glove_emds)

In [ ]:
plot_words(glove_tsne_embds, 0, 2000, 1)